In [ ]:
import asyncio
from ircclient import IRCClient
from util import parse_url

# twitch IRC server info
SERVER = 'irc.chat.twitch.tv'
CHANNEL = '#...' # set the channel you want to join
NICK = '...' # set your nickname
PASS = 'oauth:token' # you can get it from https://twitchapps.com/tmi/

#'''
from random import random

# freenode irc server example info
SERVER = 'chat.freenode.net:6666'
CHANNEL = '#hojong'
NICK = 'hjbot_%05d' % (random() * 10000)
PASS = None
#'''

# callback to print chat
def print_chat(nick, chat):
    print('*** [RECEIVED] %s: %s' % (nick, chat))
    
# callback to log to database
def log_to_db(nick, chat):
    pass

def main():
    host, port = parse_url(SERVER)
    assert (PASS != 'oauth:token'), 'you must set PASS'
    print('*** [SETTING_INFO] %s %d %s %s %s' % (host, port, CHANNEL, NICK, PASS))
    
    loop = asyncio.get_event_loop()
    if loop.is_closed():
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
        
    # if you want to use other callback, change the callback parameter
    client = IRCClient(CHANNEL, NICK, PASS, print_chat, loop)
    coroutine = loop.create_connection(lambda: client, host, port)

    try:
        loop.run_until_complete(coroutine) # connect
        # bot send chatting
        client.send_chat('hello, I am a bot!')
        loop.run_forever() # data receiving
    except KeyboardInterrupt as e:
        print(e)
    finally:
        loop.close()
    
if __name__ == '__main__':
    main()

*** [SETTING_INFO] chat.freenode.net 6666 #hojong hjbot_05249 None
*** [SENDED] NICK hjbot_05249
*** [SENDED] USER dummy 0 * :dummy
*** [SENDED] JOIN #hojong
*** [SENDED] PRIVMSG #hojong :hello I am a bot

:cherryh.freenode.net NOTICE * :*** Looking up your hostname...

:cherryh.freenode.net NOTICE * :*** Checking Ident
:cherryh.freenode.net NOTICE * :*** Couldn't look up your hostname
:cherryh.freenode.net NOTICE * :*** No Ident response

:cherryh.freenode.net 001 hjbot_05249 :Welcome to the freenode Internet Relay Chat Network hjbot_05249
:cherryh.freenode.net 002 hjbot_05249 :Your host is cherryh.freenode.net[149.56.134.238/6666], running version ircd-seven-1.1.4
:cherryh.freenode.net 003 hjbot_05249 :This server was created Wed Jan 4 2017 at 16:45:00 EST
:cherryh.freenode.net 004 hjbot_05249 cherryh.freenode.net ircd-seven-1.1.4 DOQRSZaghilopswz CFILMPQSbcefgijklmnopqrstvz bkloveqjfI
:cherryh.freenode.net 005 hjbot_05249 CHANTYPES=# EXCEPTS INVEX CHANMODES=eIbq,k,flj,CFLMPQScgimnpr